<a target="_blank" href="https://colab.research.google.com/github/ArtificialIntelligenceToolkit/aitk/blob/master/notebooks/Derivation of the Y-Combinator.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install aitk --upgrade --quiet

# Derivation of the Y-Combinator

## Douglas Blank

### Based on work by Jim Marshall

This is the derivation of the applicative-order Y-combinator from scratch, in Python. The following derivation is similar in flavor to the derivation found in The Little LISPer by Friedman/Felleisen, but uses a slightly different starting approach (for one thing, I begin with the "sum" function). Maybe this version will be a little easier to follow.

For these examples, we're going to write some little, often recursive, functions. But instead of use `def function():` we'll use lambda without any return statement, just a simple expression. 

For example, here is a named function that can add two numbers together:

In [7]:
def add(a, b):
    return a + b

And here is the same function, but this time unnamed:

In [8]:
(lambda a, b: a + b)

<function __main__.<lambda>(a, b)>

And (just like its named counterpart) here is how we can apply it to two numbers:

In [9]:
add(4, 7)

11

In [10]:
(lambda a, b: a + b)(4, 7)

11

Also, one other trick we can do to give it a name internally: we can use the default argument syntax of Python's lambda:

In [11]:
(lambda add=(lambda a, b: a + b): add(4, 7))

<function __main__.<lambda>(add=<function <lambda> at 0x7f1da7930ee0>)>

To evaluate, we call it with no arguments:

In [12]:
(lambda add=(lambda a, b: a + b): add(4, 7))()

11

And, one final trick: we can pull the default value out and pass it in:

In [13]:
(lambda add: add(4, 7))

<function __main__.<lambda>(add)>

In [14]:
(lambda add: add(4, 7))(lambda a, b: a + b)

11

Ok, with those brief examples, let's get going!

**Step 0**. We wish to write the recursive function "sum." No problem! Here is a recursive version written as one expression. It says, return 0 if `not list` (there is nothing in the list, thus length zero). Otherwise, recursively call "sum()" on everything but the first element (`sum(list[1:])`), and add the first element to that.

In [15]:
def sum(list): 
    return (0 if not list else 
            sum(list[1:]) + list[0])

In [16]:
sum([4, 7, 5, 6, 2, 4])

28

Ok, but can we write it without having to give it a name, with `lambda` and no `return`? Sorta:

In [17]:
(lambda list: 0 if not list else 
    QQQ(list[1:]) + list[0])

<function __main__.<lambda>(list)>

And as above, we could apply it to a list of numbers:

I put in a QQQ in the place of where we used the name "sum" above. Does that work? Let's apply it to a list:

In [18]:
(lambda list: 0 if not list else 
    QQQ(list[1:]) + list[0])(
        [4, 7, 5, 6, 2, 4]
)

NameError: name 'QQQ' is not defined

No. What can we replace QQQ that would work here? A copy of the function itself! Let's try replacing QQQ with the function defintion, apply it to a short list:

In [19]:
(lambda list: 0 if not list else 
    (lambda list: 0 if not list else 
         QQQ(list[1:]) + list[0])(list[1:]) + list[0])(
            [4]
)

4

It works! Well, for a list with one number in it. What about two numbers?

In [20]:
(lambda list: 0 if not list else 
   (lambda list: 0 if not list else 
        QQQ(list[1:]) + list[0])(list[1:]) + list[0])(
            [4, 7]
)

NameError: name 'QQQ' is not defined

Oh, no. We need another copy:

In [21]:
(lambda list: 0 if not list else 
   (lambda list: 0 if not list else 
        (lambda list:  0 if not list else 
             QQQ(list[1:]) + list[0])(list[1:]) 
                + list[0])(list[1:]) + list[0])(
                    [4, 7]
)

11

Yahoo! The problem, of course, is that we can't keep plugging in the function expression itself directly in place of the QQQ's forever because that immediately leads to an infinite regress. It's like trying to quote an entire sentence inside the sentence itself. 

**Step 1**. This is the only step in the entire derivation that requires any real thought. We can get around the above problem by passing in a copy of the sum function as an extra argument and then using that in place of the QQQ. But we must ensure that the copy of our function looks exactly like the function itself at all times. WE WILL ADHERE TO THIS REQUIREMENT IN EVERY STEP THAT FOLLOWS. Notice that since f is a copy of the function, and the function takes a copy of itself as a second argument, we must also pass f to f (as a second argument). Passing a copy of the function to itself is the whole secret of the Y-combinator. The following expression will evaluate to 28 (the sum of the example list): 

In [22]:
(lambda list, sum: 0 if not list else 
     sum(list[1:], sum) + list[0])(
        [4, 7, 5, 6, 2, 4], 
        (lambda list, sum: 0 if not list else 
             sum(list[1:], sum) + list[0]))

28

To keep the code abstract (and a little shorter) we'll replace "sum" with "f":

In [23]:
(lambda list, f: 0 if not list else 
     f(list[1:], f) + list[0])(
        [4, 7, 5, 6, 2, 4], 
        (lambda list, f: 0 if not list else 
             f(list[1:], f) + list[0]))

28

**Step 2**. We just switch the order of the function arguments, list and f. `f(list[1:], f)` changes to `f(f, list[1:])`, and the arguments to the top-level invocation also switch places: 

In [24]:
(lambda f, list: 0 if not list else f(f, list[1:]) + list[0])(
    (lambda f, list: 0 if not list else f(f, list[1:]) + list[0]),
    [4, 7, 5, 6, 2, 4]
)

28

**Step 3**. We simply "curry" the function so that it takes its two arguments one at a time. Note the weird f(f)(). This expression still evaluates to 28: 

In [26]:
(lambda f: lambda list: 0 if not list else f(f)(list[1:]) + list[0])(
    (lambda f: lambda list: 0 if not list else f(f)(list[1:]) + list[0]))(
        [4, 7, 5, 6, 2, 4]
    )

28

**Step 4**. The above expression is now of the form `function([4, 7, 5, 6, 2, 4])`, where `function` is now a self-contained recursive version of "sum", although still in a clumsy form. We can forget about `[4, 7, 5, 6, 2, 4]` for the remainder of the derivation and concentrate on just the `function` part, since that's what we're interested in. So here it is by itself, followed by the application:

In [27]:
(lambda f: lambda list: 0 if not list else f(f)(list[1:]) + list[0])(
    (lambda f: lambda list: 0 if not list else f(f)(list[1:]) + list[0]))

<function __main__.<lambda>.<locals>.<lambda>(list)>

And, to make sure it still works:

In [28]:
(lambda f: lambda list: 0 if not list else f(f)(list[1:]) + list[0])(
    (lambda f: lambda list: 0 if not list else f(f)(list[1:]) + list[0]))(
        [4, 7, 5, 6, 2, 4]
)

28

**Step 5**. Notice that in the above expression f(f) returns a function which gets applied to `list[1:]`. In the same way that the `add` function is equivalent to the function `(lambda a, b: add(a, b)`, the "f(f) function" is equivalent to the function `(lambda a: f(f)(a))`. [This is just an inverse eta step to you lambda-calculus pros out there]. This step is necessary to avoid infinite loops as we move forward, since we're assuming applicative order (i.e, Python). 

In [29]:
(lambda f: lambda list: 0 if not list else (lambda a: f(f)(a))(list[1:]) + list[0])(
    (lambda f: lambda list: 0 if not list else (lambda a: f(f)(a))(list[1:]) + list[0]))

<function __main__.<lambda>.<locals>.<lambda>(list)>

In [30]:
(lambda f: lambda list: 0 if not list else (lambda a: f(f)(a))(list[1:]) + list[0])(
    (lambda f: lambda list: 0 if not list else (lambda a: f(f)(a))(list[1:]) + list[0]))(
       [4, 7, 5, 6, 2, 4]
)

28

**Step 6**. Here we just give the (lambda a: f(f)(a)) function the name "r" using the default argument value. Simple. (Notice how every change to our function requires an identical change to the copy of the function, as mentioned earlier). 

In [31]:
(lambda f: lambda list, r=(lambda a: f(f)(a)): 0 if not list else r(list[1:]) + list[0])(
    (lambda f: lambda list, r=(lambda a: f(f)(a)): 0 if not list else r(list[1:]) + list[0]))

<function __main__.<lambda>.<locals>.<lambda>(list, r=<function <lambda>.<locals>.<lambda> at 0x7f1da7821940>)>

In [32]:
(lambda f: lambda list, r=(lambda a: f(f)(a)): 0 if not list else r(list[1:]) + list[0])(
    (lambda f: lambda list, r=(lambda a: f(f)(a)): 0 if not list else r(list[1:]) + list[0]))(
        [4, 7, 5, 6, 2, 4]
)

28

**Step 7**. Now we just expand the default lambda value into their equivalent lambda. In general, "(lambda x=val: body)" is equivalent to "(lambda x: body)(val)".

In [33]:
(lambda f: lambda list: (lambda r: 0 if not list else r(list[1:]) + list[0])(lambda a: f(f)(a)))(
    (lambda f: lambda list: (lambda r: 0 if not list else r(list[1:]) + list[0])(lambda a: f(f)(a))))

<function __main__.<lambda>.<locals>.<lambda>(list)>

In [34]:
(lambda f: lambda list: (lambda r: 0 if not list else r(list[1:]) + list[0])(lambda a: f(f)(a)))(
    (lambda f: lambda list: (lambda r: 0 if not list else r(list[1:]) + list[0])(lambda a: f(f)(a))))(
        [4, 7, 5, 6, 2, 4]
)

28

**Step 7b**: Rearrange the order of the curry: f, r, list:

In [35]:
(lambda f: (lambda r: (lambda list: 0 if not list else r(list[1:]) + list[0]))(
                (lambda a: f(f)(a))
))(
  (lambda f: (lambda r: (lambda list: 0 if not list else r(list[1:]) + list[0]))(
                (lambda a: f(f)(a))
  ))
)

<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>(list)>

In [36]:
(lambda f: (lambda r: (lambda list: 0 if not list else r(list[1:]) + list[0]))(
                (lambda a: f(f)(a))
))(
  (lambda f: (lambda r: (lambda list: 0 if not list else r(list[1:]) + list[0]))(
                (lambda a: f(f)(a))
  ))
)(
    [4, 7, 5, 6, 2, 4]
)

28

**Step 8**. Now we can give the (lambda r: (lambda list: ...)) expression a name also ("m") using the default lambda syntax, since it has no free variables (except for primitives, but they're bound globally anyway). This step is just like Step 6. 

In [37]:
(lambda m=(lambda r: (lambda list: 0 if not list else r(list[1:]) + list[0])):
    (lambda f: m(lambda a: f(f)(a)))(
        (lambda f: m(lambda a: f(f)(a))))
)()

<function __main__.<lambda>.<locals>.<lambda>(list)>

In [38]:
(lambda m=(lambda r: (lambda list: 0 if not list else r(list[1:]) + list[0])):
    (lambda f: m(lambda a: f(f)(a)))(
        (lambda f: m(lambda a: f(f)(a))))
)()(
 [4, 7, 5, 6, 2, 4]
)

28

**Step 9**. Now we replace the let-expression for "m" by its equivalent lambda-form, just like in Step 7, and out pops the applicative-order Y-combinator! The expression below still represents the self-contained recursive sum function, but now it's in a nicer form. In particular, the (lambda m: ...) sub-expression is Y: 

In [39]:
(lambda m:
    (lambda f: m(lambda a: f(f)(a)))(
      (lambda f: m(lambda a: f(f)(a)))
    )
)(
    lambda r: (lambda list: 0 if not list else r(list[1:]) + list[0])
)

<function __main__.<lambda>.<locals>.<lambda>(list)>

In [40]:
(lambda m:
    (lambda f: m(lambda a: f(f)(a)))(
      (lambda f: m(lambda a: f(f)(a)))
    )
)(
    lambda r: (lambda list: 0 if not list else r(list[1:]) + list[0])
)(
 [4, 7, 5, 6, 2, 4]
)

28

**Step 10**. We just pull out the (lambda m: ...) sub-expression and call it Y, since all of its variables are bound (after all, it's a combinator). Then the expression above for the recursive sum function can be rewritten as shown below. The expression passed to Y is a "template" for the recursive sum function. Instead of "QQQ", we call the recursive invocation "r", wrap the whole thing with (lambda r: ...), and hand it over to Y, which returns a self-contained recursive function. You can give it a name with define if you want, but you don't have to. 

In [41]:
def Y(m):
    return (lambda f: m(lambda a: f(f)(a)))(
                (lambda f: m(lambda a: f(f)(a)))
            )

In [42]:
Y(lambda r: (lambda list: 0 if not list else r(list[1:]) + list[0]))

<function __main__.<lambda>.<locals>.<lambda>(list)>

In [43]:
Y(lambda r: (lambda list: 0 if not list else r(list[1:]) + list[0]))([4, 7, 5, 6, 2, 4])

28

We can use Y for any recursive function, such as length:

In [44]:
Y(lambda r: (lambda list: 0 if not list else r(list[1:]) + 1))(["any", "old", "list", "you", "like"])

5

In [45]:
Y(lambda fib: (lambda number: 1 if number in [1, 2] else fib(number - 1) + fib(number - 2)))(9)

34

In [46]:
for i in range(1, 10):
    print(Y(lambda fib: (lambda number: 1 if number in [1, 2] else fib(number - 1) + fib(number - 2)))(i))

1
1
2
3
5
8
13
21
34


I hope that you have enjoyed this derivation of the Y-combinator!